### Interpolate Velocities
Read and interpolate time-variable ice velocity grids to fill data gaps

In [ ]:
import timescale
import numpy as np
import xarray as xr
import pointAdvection
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#### Read input velocity field using ``xarray`` and ``pointAdvection``

In [ ]:
# read velocity file with xarray
with xr.open_dataset('Norden.NSIDC-0731.nc') as ds:
    ts = timescale.time.Timescale().from_datetime(ds.time.data)
    adv = pointAdvection.advection(time_units='years').from_dict({
            'x':np.array(ds.x),
            'y':np.array(ds.y[::-1]), 
            'U':np.array(ds.VelocitySeries[:,0,::-1,:]),
            'V':np.array(ds.VelocitySeries[:,1,::-1,:]), 
            'eU':np.array(ds.VelocitySeries[:,3,::-1,:]),
            'eV':np.array(ds.VelocitySeries[:,4,::-1,:]),
            'time':ts.year
        },
        t_axis=0,
        scale=1.0
    )

#### Plot original velocity with data gaps

In [ ]:
# create output figure axis
fig,ax = plt.subplots(num=1, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# create color map
cmap = pointAdvection.tools.custom_colormap(180, 'Rignot')
# show velocity magnitude
adv.imshow(ax=ax[0], band=1, cmap=cmap, norm=colors.LogNorm(vmin=1, vmax=3000), alpha=0.75)
# show velocity divergence
adv.imshow(ax=ax[1], band=1, imtype='divergence', cmap=plt.cm.coolwarm, vmin=-0.1, vmax=0.1, alpha=0.75)
# tight layout
fig.tight_layout()
plt.show()

#### Calculate an error-weighted version to fill data gaps

In [ ]:
# calculate an error-weighted average of the velocities
v = adv.copy()
v.fill_velocity_gaps()

#### Plot gap-filled velocity fields

In [ ]:
# create output figure axis
fig,ax = plt.subplots(num=2, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# show velocity magnitude
v.imshow(ax=ax[0], band=1, cmap=cmap, norm=colors.LogNorm(vmin=1, vmax=3000), alpha=0.75)
# show velocity divergence
v.imshow(ax=ax[1], band=1, imtype='divergence', cmap=plt.cm.coolwarm, vmin=-0.1, vmax=0.1, alpha=0.75)
# tight layout
fig.tight_layout()
plt.show()